In [1]:
from collections import defaultdict
from itertools import cycle
from tqdm import tqdm

In [2]:
class CaveTetris():
    def __init__(self, gas_stream: list[int], cave_width: int = 7) -> None:
        self.gas_stream = gas_stream
        self.cave_width = cave_width
        self.max_height = 0
        self.blocks = [
            Block([(0, 0), (1, 0), (2, 0), (3, 0)]),
            Block([(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)]),
            Block([(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)]),
            Block([(0, 0), (0, 1), (0, 2), (0, 3)]),
            Block([(0, 0), (0, 1), (1, 0), (1, 1)]),
        ]
        self._state_dict = defaultdict(int)

        self._char_rep = {
            0: ".",
            1: "–",
            2: "|",
            3: "+",
            4: "#",
            -2: "@"
        }

        for _x in range(cave_width):
            self[(_x, 0)] = 1

    def __getitem__(self, key: tuple[int, int]) -> int:
        if key[1] <= 0 and (key[0] < 0 or key[0] > self.cave_width):
            return 3
        elif key[1] <= 0:
            return 1
        elif key[0] < 0 or key[0] >= self.cave_width:
            return 2
        else:
            return self._state_dict[key]

    def __setitem__(self, key: tuple[int, int], value: int) -> None:
        self._state_dict[key] = value
    
    def simultate_falling_rocks(self, num_of_blocks: int, verbose: bool = False):
        block_gen = enumerate(cycle(self.blocks))
        curr_block = None
        cycle_store = {}
        height_store = {}
        for stream_idx, stream in cycle(enumerate(self.gas_stream)):
            if curr_block is None:
                block_idx, curr_block = next(block_gen)
                height_store[block_idx] = self.max_height
                if block_idx >= num_of_blocks:
                    break
                curr_block.reset(self.max_height)

                curr_height = []
                for _x in range(self.cave_width):
                    for _y in range(self.max_height, -1, -1):
                        if self[(_x, _y)] > 0:
                            curr_height.append(_y - self.max_height)
                            break
                curr_height = tuple(curr_height)
                if (block_idx % 5, curr_height, stream_idx) not in cycle_store:
                    cycle_store[(block_idx % 5, curr_height, stream_idx)] = (self.max_height, block_idx)
                else:
                    past_max_height, past_block_idx = cycle_store[(block_idx % 5, curr_height, stream_idx)]
                    print(f"FOUND: {past_max_height=}, {past_block_idx=}, {self.max_height=}, {block_idx=}")
                    cycle_length = block_idx - past_block_idx
                    cycle_height = self.max_height - past_max_height

                    print(f"{cycle_length=}, {cycle_height=}")
                    remaining_blocks = num_of_blocks - block_idx
                    remaining_cycles = remaining_blocks // cycle_length
                    remaining_height = (remaining_blocks // cycle_length) * cycle_height
                    incomplete_cycle = remaining_blocks % cycle_length
                    print(f"{remaining_cycles=}: {remaining_height}")
                    incomplete_cycle_height = height_store[past_block_idx + incomplete_cycle] - past_max_height
                    print(f"{incomplete_cycle=}: {incomplete_cycle_height}")

                    print(f"Alltogether: {self.max_height + remaining_height + incomplete_cycle_height=}")
                    return self.max_height + remaining_height + incomplete_cycle_height

            if verbose:
                self.fix_block(curr_block, temporary=True)
                print(self)
            curr_block.move_sideways(self, stream)
            if verbose:
                self.fix_block(curr_block, temporary=True)
                print(self)
            if not curr_block.move_down(self):
                self.max_height = max(self.max_height, curr_block.y + curr_block.height)
                self.fix_block(curr_block)
                curr_block = None
            elif verbose:
                self.fix_block(curr_block, temporary=True)
            if verbose:
                print(self)
        return self.max_height
    
    def fix_block(self, block: "Block", temporary: bool = False) -> None:
        # for k, v in self._state_dict.items():
        #     if v < 0:
        #         self[k] = 0
        for x, y in block.rock_positions:
            self[block.x + x, block.y + y] = -2 if temporary else 4

    def __iter__(self):
        return iter(self._state_dict)

    def __repr__(self) -> str:
        min_y = min(k[1] for k in self)
        max_y = max(k[1] for k in self)

        ret_str = ""
        for y in range(max_y, min_y - 1, -1):
            ret_str += f"{y:3d} "
            for x in range(-1, self.cave_width + 1):
                ret_str += self._char_rep[self[(x, y)]]
            ret_str += "\n"
        return ret_str
            
class Block():
    def __init__(self, rock_positions: list[tuple[int, int]]) -> None:
        self.rock_positions = rock_positions
        self.width = max(pos[0] for pos in rock_positions)
        self.height = max(pos[1] for pos in rock_positions)

    def move(self, cave: CaveTetris, step: tuple[int, int]) -> bool:
        step_x, step_y = step
        for rock_x, rock_y in self.rock_positions:
            if cave[(self.x + rock_x + step_x, self.y + rock_y + step_y)] > 0:
                return False
        self.x += step_x
        self.y += step_y
        return True

    def move_down(self, cave: CaveTetris) -> bool:
        return self.move(cave, (0, -1))

    def move_sideways(self, cave: CaveTetris, step: int) -> bool:
        return self.move(cave, (step, 0))
    
    def reset(self, curr_max_height: int) -> None:
        self.x = 2
        self.y = curr_max_height + 4


In [3]:
def parse_input(input_gen):
    row = next(input_gen).rstrip()
    return CaveTetris([
        1 if a == ">" else -1
        for a in row
    ])

In [4]:
def task_one(filename: str, num_blocks: int = 2022) -> int:
    with open(filename) as f:
        cave = parse_input(f)
    return cave.simultate_falling_rocks(num_blocks)

In [5]:
task_one("test-input.txt")

FOUND: past_max_height=49, past_block_idx=28, self.max_height=102, block_idx=63
cycle_length=35, cycle_height=53
remaining_cycles=55: 2915
incomplete_cycle=34: 51
Alltogether: self.max_height + remaining_height + incomplete_cycle_height=3068


3068

In [6]:
task_one("input.txt")

FOUND: past_max_height=363, past_block_idx=235, self.max_height=2935, block_idx=1945
cycle_length=1710, cycle_height=2572
remaining_cycles=0: 0
incomplete_cycle=77: 113
Alltogether: self.max_height + remaining_height + incomplete_cycle_height=3048


3048

In [7]:
task_one("test-input.txt", num_blocks=1000000000000)

FOUND: past_max_height=49, past_block_idx=28, self.max_height=102, block_idx=63
cycle_length=35, cycle_height=53
remaining_cycles=28571428569: 1514285714157
incomplete_cycle=22: 29
Alltogether: self.max_height + remaining_height + incomplete_cycle_height=1514285714288


1514285714288

In [8]:
task_one("input.txt", num_blocks=1000000000000)

FOUND: past_max_height=363, past_block_idx=235, self.max_height=2935, block_idx=1945
cycle_length=1710, cycle_height=2572
remaining_cycles=584795320: 1504093563040
incomplete_cycle=855: 1274
Alltogether: self.max_height + remaining_height + incomplete_cycle_height=1504093567249


1504093567249